In [4]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
housing = fetch_california_housing()

In [5]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [6]:
x = housing.data
y = housing.target

In [7]:
from sklearn.model_selection import train_test_split
x_train_full,x_test,y_train_full,y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full,y_train_full, random_state = 42)
len(x_train_full)

16512

In [8]:
len(x_train), len(x_valid)

(12384, 4128)

In [9]:
x_train.shape[1]

8

In [10]:
x_train.shape[1:]

(8,)

In [11]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test  = scaler.transform(x_test)

In [12]:
import tensorflow as tf
from tensorflow import keras

In [65]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

In [119]:
keras.backend.clear_session()
tf.random.set_seed(42)

In [120]:
model = keras.models.Sequential([keras.layers.Dense(30, activation = 'relu', input_shape = [x_train.shape[1]]),
                                 keras.layers.Dense(15,activation = 'relu'),
                                 keras.layers.Dense(1)])

In [121]:
model.compile(loss = 'mse', optimizer = keras.optimizers.SGD(lr = 1e-3))
#history = model.fit(x_train,y_train, epochs = 20, validation_data=(x_valid,y_valid),)

In [122]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint('checkpoint.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3, restore_best_weights=True, min_delta = 0.01)
history = model.fit(x_train, y_train, epochs=50,validation_data=(x_valid,y_valid), callbacks=[tensorboard_cb,checkpoint_cb,early_stopping_cb])

Epoch 1/50
387/387 [==============================] - 1s 3ms/step - loss: 4.1180 - val_loss: 1.0623
Epoch 2/50
387/387 [==============================] - 0s 689us/step - loss: 0.8267 - val_loss: 0.6761
Epoch 3/50
387/387 [==============================] - 0s 695us/step - loss: 0.6257 - val_loss: 0.6212
Epoch 4/50
387/387 [==============================] - 0s 697us/step - loss: 0.5950 - val_loss: 0.5890
Epoch 5/50
387/387 [==============================] - 0s 676us/step - loss: 0.5636 - val_loss: 0.5633
Epoch 6/50
387/387 [==============================] - 0s 688us/step - loss: 0.5421 - val_loss: 0.5411
Epoch 7/50
 89/387 [=====>........................] - ETA: 0s - loss: 0.4985

KeyboardInterrupt: 

In [19]:
mse_test = model.evaluate(x_test,y_test)

129/129 [==============================] - 0s 460us/step - loss: 0.4170


In [14]:
input_ = keras.layers.Input(shape=x_train.shape[1:])
hidden1=keras.layers.Dense(30,activation="relu")(input_)
hidden2=keras.layers.Dense(30,activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_,hidden2])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

In [15]:
input_A = keras.layers.Input(shape=[5],name="wide_input")
input_B = keras.layers.Input(shape=[6],name="deep_input")
hidden1 = keras.layers.Dense(30,activation="relu")(input_B)
hidden2 = keras.layers.Dense(30,activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model = keras.Model(inputs=[input_A, input_B],outputs=[output])

In [17]:
model.compile(loss="mse",optimizer=keras.optimizers.SGD(lr=1e-3))

In [19]:
x_train_A, x_train_B = x_train[:, :5], x_train[:, 2:]
x_valid_A, x_valid_B = x_valid[:, :5], x_valid[:, 2:]
x_test_A,  x_test_B  = x_test[:, :5],  x_test[:, 2:]
x_new_A,   x_new_B   = x_test_A[:3],   x_test_B[:3]
history = model.fit((x_train_A, x_train_B), y_train,epochs=20,validation_data=((x_valid_A,x_valid_B),y_valid))

Epoch 1/20
387/387 [==============================] - 1s 1ms/step - loss: 3.1020 - val_loss: 0.9717
Epoch 2/20
387/387 [==============================] - 0s 744us/step - loss: 0.8325 - val_loss: 0.7239
Epoch 3/20
387/387 [==============================] - 0s 776us/step - loss: 0.6724 - val_loss: 0.6534
Epoch 4/20
387/387 [==============================] - 0s 834us/step - loss: 0.6423 - val_loss: 0.6121
Epoch 5/20
387/387 [==============================] - 0s 767us/step - loss: 0.6045 - val_loss: 0.5841
Epoch 6/20
387/387 [==============================] - 0s 775us/step - loss: 0.5771 - val_loss: 0.5634
Epoch 7/20
387/387 [==============================] - 0s 870us/step - loss: 0.5494 - val_loss: 0.5473
Epoch 8/20
387/387 [==============================] - 0s 724us/step - loss: 0.5293 - val_loss: 0.5329
Epoch 9/20
387/387 [==============================] - 0s 704us/step - loss: 0.5134 - val_loss: 0.5216
Epoch 10/20
387/387 [==============================] - 0s 706us/step - loss: 0.5124 

In [20]:
mse_test = model.evaluate((x_test_A, x_test_B),y_test)
y_pred   = model.predict(( x_new_A,  x_new_B))

129/129 [==============================] - 0s 434us/step - loss: 0.4742


In [31]:
output = keras.layers.Dense(1,name="main_output")(concat)
aux_output = keras.layers.Dense(1,name="aux_output")(hidden2)
model = keras.Model(inputs=[input_A, input_B],outputs=[output, aux_output])
model.compile(loss=["mse", "mse"], loss_weights=[0.9,0.1], optimizer="sgd")

In [32]:
history = model.fit( [x_train_A, x_train_B],[y_train, y_train], epochs=20,validation_data=([x_valid_A, x_valid_B],[y_valid,y_valid]))

Epoch 1/20
387/387 [==============================] - 1s 2ms/step - loss: 1.1760 - main_output_loss: 0.9790 - aux_output_loss: 2.9488 - val_loss: 0.4128 - val_main_output_loss: 0.3726 - val_aux_output_loss: 0.7742
Epoch 2/20
387/387 [==============================] - 0s 1ms/step - loss: 0.3835 - main_output_loss: 0.3542 - aux_output_loss: 0.6472 - val_loss: 0.3845 - val_main_output_loss: 0.3561 - val_aux_output_loss: 0.6408
Epoch 3/20
387/387 [==============================] - 0s 1ms/step - loss: 0.3766 - main_output_loss: 0.3562 - aux_output_loss: 0.5605 - val_loss: 0.3742 - val_main_output_loss: 0.3491 - val_aux_output_loss: 0.6008
Epoch 4/20
387/387 [==============================] - 0s 894us/step - loss: 0.3624 - main_output_loss: 0.3422 - aux_output_loss: 0.5440 - val_loss: 0.3675 - val_main_output_loss: 0.3442 - val_aux_output_loss: 0.5772
Epoch 5/20
387/387 [==============================] - 0s 973us/step - loss: 0.3617 - main_output_loss: 0.3444 - aux_output_loss: 0.5170 - val_

In [27]:
mse_test = model.evaluate((x_test_A,  x_test_B),(y_test,y_test))
y_pred_main, y_pred_aux   = model.predict(( x_new_A,  x_new_B))

129/129 [==============================] - 0s 650us/step - loss: 0.3729 - main_output_loss: 0.3603 - aux_output_loss: 0.4867


In [28]:
y_pred_main, y_pred_aux

(array([[0.40951228],
        [1.5830526 ],
        [3.5901754 ]], dtype=float32),
 array([[0.8465092],
        [1.9512239],
        [2.89537  ]], dtype=float32))

In [26]:
total_loss, main_loss ,aux_loss = model.evaluate((x_test_A,  x_test_B),(y_test,y_test))

129/129 [==============================] - 0s 763us/step - loss: 0.3729 - main_output_loss: 0.3603 - aux_output_loss: 0.4867


In [34]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 30)           930         dense_5[0][0]                    
____________________________________________________________________________________________

# Saving a model

In [39]:
model.save("test_model.h5")

In [123]:
modell = keras.models.load_model('checkpoint.h5')

In [124]:
history = model.fit(x_train, y_train, epochs=50,validation_data=(x_valid,y_valid), )

Epoch 1/50
387/387 [==============================] - 0s 747us/step - loss: 0.5092 - val_loss: 0.5157
Epoch 2/50
387/387 [==============================] - 0s 730us/step - loss: 0.4956 - val_loss: 0.5034
Epoch 3/50
387/387 [==============================] - 0s 736us/step - loss: 0.4848 - val_loss: 0.4917
Epoch 4/50
387/387 [==============================] - 0s 818us/step - loss: 0.4758 - val_loss: 0.4836
Epoch 5/50
387/387 [==============================] - 0s 796us/step - loss: 0.4692 - val_loss: 0.4789
Epoch 6/50
387/387 [==============================] - 0s 752us/step - loss: 0.4629 - val_loss: 0.4740
Epoch 7/50
387/387 [==============================] - 0s 761us/step - loss: 0.4573 - val_loss: 0.4641
Epoch 8/50
387/387 [==============================] - 0s 720us/step - loss: 0.4528 - val_loss: 0.4603
Epoch 9/50
387/387 [==============================] - 0s 761us/step - loss: 0.4477 - val_loss: 0.4563
Epoch 10/50
387/387 [==============================] - 0s 878us/step - loss: 0.443

callbacks=[tensorboard_cb,checkpoint_cb,early_stopping_cb]